***Disclaimer***: This file referenced some files in other directories. In order to have working cross referencing it's recommended to start the notebook server from the root directory (`Grid2Op`) of the package and __not__ in the `getting_started` sub directory:
```bash
cd Grid2Op
jupyter notebook
```

***NB*** For more information about how to use the package, a general help can be built locally (provided that sphinx is installed on the machine) with:
```bash
cd Grid2Op
make html
```
from the top directory of the package (usually `Grid2Op`).

Once build, the help can be access from [here](../documentation/html/index.html)

**Objectives**

TODO !!!

In [1]:
import os
import sys
import grid2op
from grid2op.Agent import DoNothingAgent, Agent
from tqdm.notebook import tqdm
import numpy as np
max_iter = 10  # to make computation much faster we will only consider 50 time steps instead of 287

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/donnotben/.local/lib/python3.6/site-packages/pandapower/io_utils.py:8: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_series_equal, assert_frame_equal


In [2]:
res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

## What is redispatching

TODO

## How to implement redispatching actions

### Having a suitable environment

In [3]:
env_wrong = grid2op.make("case5_example")

[0. 0.]


In [4]:
env_wrong.redispatching_unit_commitment_availble

False

TODO + explain what is needed

In [5]:
env = grid2op.make("case14_redisp")

/home/donnotben/.local/lib/python3.6/site-packages/grid2op/MakeEnv.py:339: UserWarning: Your are using only 2 chronics for this environment. More can be download with TODO
  warnings.warn("Your are using only 2 chronics for this environment. More can be download with TODO")


[0. 0. 0. 0. 0.]


In [6]:
env.redispatching_unit_commitment_availble

True

blablabla everyting ok. We don't really care about the losses here, but rather the economic cost of the system. This is implemented in the specific reward blablabla

also, we have the information about which generators are dispatchable blablabla

### More case for ambiguous actions

In [7]:
env.gen_redispatchable

array([ True,  True, False, False,  True])

In [8]:
env.gen_cost_per_MW

array([ 5., 10.,  0.,  0., 10.])

blabla

new kind of ambiguous action

In [9]:
act = env.action_space({"redispatch": [(2,+10)]})
act.is_ambiguous()

(True,
 Grid2OpException AmbiguousAction InvalidRedispatching InvalidRedispatching('Trying to apply a redispatching action on a non redispatchable generator',))

In [10]:
act = env.action_space({"redispatch": [(0,+10)]})
act.is_ambiguous()

(True,
 Grid2OpException AmbiguousAction InvalidRedispatching InvalidRedispatching('Some redispatching amount are above the maximum ramp up',))

In [11]:
act = env.action_space({"redispatch": [(1,+10)]})
act.is_ambiguous()

(False, None)

### The setpoint is not the implementation

must ensure everything sum to 0 (automatic)

Redispatching actions stack => explain with env.step

Canceling an action is explicit

### A different reward

### A continuous problem

In [12]:
agent = DoNothingAgent(env.action_space)
done = False
reward = env.reward_range[0]

env.set_id(0)
obs = env.reset()
cum_reward = 0
nrow = env.chronics_handler.max_timestep() if max_iter <= 0 else max_iter
gen_p = np.zeros((nrow, env.n_gen))
gen_p_setpoint = np.zeros((nrow, env.n_gen))
load_p = np.zeros((nrow, env.n_load))
rho = np.zeros((nrow, env.n_line))
i = 0
with tqdm(total=max_iter, desc="step") as pbar:
    while not done:
        act = agent.act(obs, reward, done)
        obs, reward, done, info = env.step(act)
        data_generator = env.chronics_handler.real_data.data
        gen_p_setpoint[i,:] = data_generator.prod_p[data_generator.current_index, :]
        gen_p[i,:] = obs.prod_p
        load_p[i,:] = obs.load_p
        rho[i,:] = obs.rho
        cum_reward += reward
        i += 1
        pbar.update(1)
        if i >= max_iter:
            break

[0. 0. 0. 0. 0.]


[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]



In [13]:
cum_reward

73682.94396859208

let's do the same, but forcing as much redispatching as possible on the cheapest generator.

In [14]:
class GreedyEconomic(Agent):
    def __init__(self, action_space):
        Agent.__init__(self, action_space)
        self.do_nothing = action_space()
        
    def act(self, obs, reward, done):
        act = self.do_nothing
        if obs.prod_p[0] < obs.gen_pmax[0] - 1e-1:
            # if the cheapest generator is significantly bellow its maximum cost
            if obs.target_dispatch[0] < obs.gen_pmax[0]:
                #in theory i can still ask for more
                act = env.action_space({"redispatch": [(0, obs.gen_max_ramp_up[0])]})
        return act
    
agent = GreedyEconomic(env.action_space)
done = False
reward = env.reward_range[0]

env.set_id(0) # reset the env to the same id
obs = env.reset()
cum_reward = 0
nrow = env.chronics_handler.max_timestep() if max_iter <= 0 else max_iter
gen_p = np.zeros((nrow, env.n_gen))
gen_p_setpoint = np.zeros((nrow, env.n_gen))
load_p = np.zeros((nrow, env.n_load))
rho = np.zeros((nrow, env.n_line))
i = 0
with tqdm(total=max_iter, desc="step") as pbar:
    while not done:
        act = agent.act(obs, reward, done)
        obs, reward, done, info = env.step(act)
        data_generator = env.chronics_handler.real_data.data
        gen_p_setpoint[i,:] = data_generator.prod_p[data_generator.current_index, :]
        gen_p[i,:] = obs.prod_p
        load_p[i,:] = obs.load_p
        rho[i,:] = obs.rho
        cum_reward += reward
        i += 1
        pbar.update(1)
        if i >= max_iter:
            break

[0. 0. 0. 0. 0.]


[ 4.7911565  -2.39557825  0.          0.         -2.39557825]
[10.  -2.5  0.   0.  -2.5]



InvalidRedispatching: Grid2OpException AmbiguousAction InvalidRedispatching "Some redispatching amount are above the maximum ramp up"